## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [2]:
df = pd.read_csv('map_NorthMiamiBeach_popup_pin - Sheet1.csv')

## Image URL collector

In [3]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [4]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [5]:
len(links)

10

## Article link formatter

In [6]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/08...
1    <a href="https://therealdeal.com/miami/2022/08...
2    <a href="https://therealdeal.com/miami/2020/12...
3    <a href="https://therealdeal.com/miami/2022/08...
4    <a href="https://therealdeal.com/miami/2022/08...
5    <a href="https://therealdeal.com/miami/2022/07...
6    <a href="https://therealdeal.com/miami/2022/03...
7    <a href="https://therealdeal.com/miami/2022/03...
8    <a href="https://therealdeal.com/miami/2022/04...
9    <a href="https://therealdeal.com/miami/2021/04...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['geo_address'] = df['full_address'] + ' North Miami Beach, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [9]:
df.columns

Index(['full_address', 'developers', 'units', 'apts_condos', 'land_sale',
       'loan_amount', 'description', 'story_link', 'story_link_2',
       'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [10]:
df

,full_address,developers,units,apts_condos,land_sale,loan_amount,description,story_link,story_link_2,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,16955-17071 West Dixie Highway,Trinsic Residential Group,373.0,apartments,18000000.0,99000000.0,Trinsic Residential paid roughly $18 million f...,https://therealdeal.com/miami/2022/08/16/trins...,NaN,https://s14.therealdeal.com/trd/m/up/2022/08/T...,Trinsic Residential,paid roughly $18 million for a five-lot assem...,"<a href=""https://therealdeal.com/miami/2022/08...",16955-17071 West Dixie Highway North Miami Bea...,"(16955 W Dixie Hwy, North Miami Beach, FL 3316...","(25.932546, -80.152974, 0.0)",25.932546,-80.152974,0.0
1,16375 Biscayne Boulevard,Estate Companies,363.0,apartments,NaN,NaN,"The Estate Companies is planning a 28-story, 3...",https://therealdeal.com/miami/2022/08/15/estat...,NaN,https://s11.therealdeal.com/trd/m/up/2022/08/m...,The Estate Companies,"is planning a 28-story, 363-unit rental tower...","<a href=""https://therealdeal.com/miami/2022/08...","16375 Biscayne Boulevard North Miami Beach, FL","(16375 Biscayne Blvd, Aventura, FL 33160, USA,...","(25.9280335, -80.1522347, 0.0)",25.928034,-80.152235,0.0
2,16395 Biscayne Boulevard,Estate Companies,367.0,apartments,13100000.0,NaN,The Estate Companies is building a 23-story ap...,https://therealdeal.com/miami/2020/12/14/estat...,NaN,https://s11.therealdeal.com/trd/m/up/2020/12/1...,The Estate Companies,is building a 23-story apartment tower with 3...,"<a href=""https://therealdeal.com/miami/2020/12...","16395 Biscayne Boulevard North Miami Beach, FL","(16395 Biscayne Blvd, North Miami Beach, FL 33...","(25.9285698, -80.15203, 0.0)",25.928570,-80.152030,0.0
3,2261 Northeast 164th Street,BH Group,400.0,apartments,11000000.0,NaN,"BH Group plans a 30-story, 400-unit apartment ...",https://therealdeal.com/miami/2022/08/16/bh-pl...,NaN,https://s14.therealdeal.com/trd/m/up/2022/08/M...,BH Group plans a,"30-story, 400-unit apartment tower at 2261 No...","<a href=""https://therealdeal.com/miami/2022/08...","2261 Northeast 164th Street North Miami Beach, FL","(2261 NE 164th St, North Miami Beach, FL 33160...","(25.9274746, -80.1547639, 0.0)",25.927475,-80.154764,0.0
4,4098 Northeast 167 Street,Macken Companies,NaN,NaN,NaN,16500000.0,Macken Companies inked a $16.5 million constru...,https://therealdeal.com/miami/2022/08/01/macke...,https://therealdeal.com/miami/2020/10/26/macke...,https://s13.therealdeal.com/trd/m/up/2022/08/M...,Macken Companies,inked a $16.5 million construction loan from ...,"<a href=""https://therealdeal.com/miami/2022/08...","4098 Northeast 167 Street North Miami Beach, FL","(4098 NE 167th St, North Miami Beach, FL 33160...","(25.9336827, -80.1316907, 0.0)",25.933683,-80.131691,0.0
5,1750 and 1775 Northeast 167th Street,Ari Pearl’s PPG Development. Matt Press of Equ...,700.0,apartments,31100000.0,NaN,"Ari Pearl’s PPG Development, along with partne...",https://therealdeal.com/miami/2022/07/20/ari-p...,NaN,https://s11.therealdeal.com/trd/m/up/2022/07/m...,Ari Pearl’s PPG,"Development, along with partners Matt Press o...","<a href=""https://therealdeal.com/miami/2022/07...",1750 and 1775 Northeast 167th Street North Mi...,"(1775 NE 167th St, North Miami Beach, FL 33162...","(25.9298195, -80.1652049, 0.0)",25.929820,-80.165205,0.0
6,Southeast corner of Northeast 159th Street and...,AHS Residential,NaN,NaN,57000000.0,NaN,Resia (formerly AHS Residential) paid $57 mill...,https://therealdeal.com/miami/2022/03/28/ahs-r...,NaN,https://s14.therealdeal.com/trd/m/up/2022/03/A...,Resia (formerly AHS,Residential) paid $57 million for the develop...,"<a href=""https://therealdeal.com/miami/2022/03...",Southeast corner of Northeast 159th Street and...,"(FL-909 & NE 159th St, North Miami Beach, FL 3...","(25.9224504, -80.15588350000002, 0.0)",25.922450,-80.155884,0.0
7,2050 Northeast 164th Street,Carpe Real Estate Partners,440.0,apartments,NaN,NaN,Oak Row Equities (

## Correction section

In [11]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [12]:
df.columns

Index(['full_address', 'developers', 'units', 'apts_condos', 'land_sale',
       'loan_amount', 'description', 'story_link', 'story_link_2',
       'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [13]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developer) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [14]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [15]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/08/16/trinsic-nabs-99m-loan-to-build-north-miami-beach-rentals/" target="_blank" rel="noopener noreferrer">Trinsic Residential</a> paid roughly $18 million for a five-lot assemblage in March from an affiliate of North Miami Beach-based Macken Companies, with plans to build a 373-unit apartment complex and 17,000 square feet of retail. This month, the developers scored a $99 million construction loan from TD Bank for the project. Macken retained ownership of the commercial part of the project.'

In [16]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'North Miami Beach Resi Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [17]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [18]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [19]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/north_miami_beach
